In [24]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 29 09:31:48 2018

@author: eduardo-ssr
"""
import sys
import serial
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 
from drawnow import drawnow
from time import sleep
from datetime import datetime
import timeit
from scipy import signal

%matplotlib inline
read0 = []
read1 = []
read2 = []
read3 = []
read4 = []
read5 = []
read6 = []
read7 = []

acelX = []
acelY = []
acelZ = []
gyroX = []
gyroY = []
gyroZ = []

fx = []
#constant_Calib_Acel = (16384/9.81)
constant_Calib_Acel = (16384/9.81)
constant_Calib_Gyro = 131


os.system('clear')
print ("======= Lista de dispositivos USB =======")
serial_ports = glob.glob('/dev/ttyUSB*')
while(len(serial_ports)==0):
    print ("Conecte o Arduino...\n")
    sleep(5);
    serial_ports = glob.glob('/dev/ttyUSB*')
        
for i in range(len(serial_ports)): 
    print (i, " - ", serial_ports[i])
    port = input("Escolha a porta do Arduino (e.g. 0): ")
        
ser = serial.Serial(serial_ports[int(port)], 2000000,timeout=1)

def main():
    global contador
    global t
    global constant_Calib_Acel
    global constant_Calib_Gyro
    
    if ser.is_open:
        print("Comunicação Serial Estabelecida.\n")
    else :
        print("Erro na Comunicação Serial.\n")
        
        
    input("Pressione 'Enter' para iniciar a leitura e \"I\" duas vezes para pausar: \n")
    
    
    i = 0
    inicio = timeit.default_timer()

    while(True):    
        try:
            
            line = ser.readline().decode("utf-8")
            print(line)
            
            try:
                entry = line.split("\t")
                if len(entry) != 8 :
                    print('Erro de valor.')
                else:    
                    n = np.int(entry[0])
                    AcX = np.float(entry[1])
                    AcY = np.float(entry[2])
                    AcZ = np.float(entry[3])
                    temp = np.float(entry[4])
                    Gx = np.float(entry[5])
                    Gy = np.float(entry[6])
                    Gz = np.float(entry[7])

                    ACX = AcX/constant_Calib_Acel  # entre--20 m/s² e + 20 m/s² 
                    ACY = AcY/constant_Calib_Acel  # entre--20 m/s² e + 20 m/s² 
                    ACZ = AcZ/constant_Calib_Acel  # entre--20 m/s² e + 20 m/s² 

                    Temp = temp/340.00 + 36.53

                    GX = Gx/constant_Calib_Gyro # entre +250º/s e -250º/s
                    GY = Gy/constant_Calib_Gyro # entre +250º/s e -250º/s
                    GZ = Gz/constant_Calib_Gyro # entre +250º/s e -250º/s
                    
                    read0.append(n)
                    read1.append(AcX) 
                    read2.append(AcY)
                    read3.append(AcZ)
                    read4.append(Gx)
                    read5.append(Gy)
                    read6.append(Gz)
                    read7.append(temp)

                    acelX.append(ACX)
                    acelY.append(ACY)
                    acelZ.append(ACZ)
                    gyroX.append(GX)
                    gyroY.append(GY)
                    gyroZ.append(GZ)
                    
                    fx.append(i) 
                    i = i+1
                    
            except (ValueError):
                print("Erro de valor.")
                pass
        
        except (KeyboardInterrupt):
            fim = timeit.default_timer()
            tempo = fim - inicio
            now = datetime.now()
            
            print('Tempo médio de', i ,'execuções foi de ',tempo/i,' s', '\n')
            print('O tempo total foi:',tempo)
            
            print ("Voce interrompeu este programa.\n")

            ser.close()
            
            
            x = np.vstack((read0, read1,read2,read3,read4,read5,read6,read7))
            y = np.vstack((read0, acelX,acelY,acelZ,gyroX,gyroY,gyroZ))
            
            r = input('Deseja salvar os dados adquiridos (S/N)? ')
            if r == 'S' or r == 's':
            
                np.savetxt('Dados_Brutos_%s.csv'%(str(now)[:-7]), np.transpose(x), delimiter=';')
                np.savetxt('Dados_Fisicos_%s.csv'%(str(now)[:-7]), np.transpose(y), delimiter=';') 
                print ('Seus dados foram salvos como: Dados_Fisicos_%s.csv'%(str(now)[:-7]))
            
            
            break
        
            
if __name__ == "__main__":
    main()


======= Lista de dispositivos USB =======
0  -  /dev/ttyUSB0
Escolha a porta do Arduino (e.g. 0): 0
Comunicação Serial Estabelecida.

Pressione 'Enter' para iniciar a leitura e "I" duas vezes para pausar: 

.00	-1069.00	-17302.00	94.00

Erro de valor.
2	-3704.00	-204.00	14172.00	-2448.00	187.00	-241.00	-140.00

3	-2728.00	-4068.00	15788.00	-2384.00	-742.00	-102.00	-112.00

1	-9856.00	-32768.00	11300.00	-3248.00	-1007.00	-17237.00	125.00

2	-3792.00	-60.00	14216.00	-2432.00	20.00	-266.00	-51.00

3	-2760.00	-4176.00	15804.00	-2432.00	-701.00	-108.00	-96.00

1	-9768.00	-32768.00	11348.00	-3200.00	-1057.00	-17286.00	82.00

2	-3872.00	-104.00	14068.00	-2400.00	178.00	-254.00	-124.00

3	-2700.00	-4184.00	15876.00	-2432.00	-683.00	-69.00	-88.00

1	-9824.00	-32768.00	11256.00	-3216.00	-1025.00	-17277.00	105.00

2	-3744.00	-124.00	14064.00	-2464.00	93.00	-274.00	-66.00

3	-2720.00	-4100.00	15788.00	-2448.00	-713.00	-120.00	-112.00

1	-9804.00	-32768.00	11352.00	-3184.00	-1055.00	-17269.00	95.00

3	-2828.00	-4076.00	15704.00	-2400.00	-699.00	-41.00	-73.00

1	-9892.00	-32768.00	11232.00	-3168.00	-1022.00	-17264.00	88.00

2	-3684.00	-88.00	14220.00	-2432.00	135.00	-242.00	-123.00

3	-2776.00	-3996.00	15752.00	-2400.00	-707.00	-107.00	-95.00

1	-9924.00	-32768.00	11304.00	-3216.00	-1033.00	-17269.00	125.00

2	-3812.00	-32.00	14200.00	-2416.00	121.00	-264.00	-98.00

3	-2772.00	-4104.00	15820.00	-2432.00	-706.00	-95.00	-116.00

1	-9904.00	-32768.00	11440.00	-3232.00	-1041.00	-17287.00	127.00

2	-3792.00	-104.00	14092.00	-2432.00	122.00	-214.00	-88.00

3	-2856.00	-4132.00	15728.00	-2416.00	-695.00	-90.00	-108.00

1	-9752.00	-32768.00	11332.00	-3200.00	-1028.00	-17280.00	128.00

2	-3764.00	-128.00	14164.00	-2400.00	116.00	-247.00	-92.00

3	-2764.00	-4120.00	15732.00	-2416.00	-707.00	-80.00	-86.00

1	-9852.00	-32768.00	11388.00	-3232.00	-1029.00	-17281.00	82.00

2	-3768.00	-152.00	14124.00	-2432.00	128.00	-251.00	-128.00

3	-2764.00	-4136.00	15800.00	-2400.00	-713.00	-96.00	-105.00

1	

2	-3740.00	-236.00	14088.00	-2432.00	140.00	-249.00	-96.00

3	-2656.00	-4096.00	15764.00	-2432.00	-711.00	-66.00	-104.00

1	-9956.00	-32768.00	11184.00	-3232.00	-1030.00	-17258.00	128.00

2	-3844.00	-108.00	14224.00	-2400.00	113.00	-255.00	-112.00

3	-2728.00	-4044.00	15812.00	-2400.00	-692.00	-90.00	-128.00

1	-9960.00	-32768.00	11388.00	-3200.00	-1039.00	-17263.00	104.00

2	-3760.00	-108.00	14164.00	-2432.00	118.00	-229.00	-115.00

3	-2768.00	-4180.00	15760.00	-2432.00	-694.00	-96.00	-113.00

1	-9932.00	-32768.00	11272.00	-3232.00	-1016.00	-17276.00	117.00

2	-3644.00	-112.00	14120.00	-2432.00	118.00	-249.00	-90.00

3	-2600.00	-4204.00	15736.00	-2400.00	-676.00	-74.00	-90.00

1	-9908.00	-32768.00	11372.00	-3200.00	-1025.00	-17292.00	105.00

2	-3696.00	-144.00	14216.00	-2448.00	130.00	-245.00	-97.00

3	-2744.00	-3984.00	15772.00	-2432.00	-684.00	-87.00	-104.00

1	-9860.00	-32768.00	11232.00	-3232.00	-1054.00	-17294.00	109.00

2	-3816.00	-136.00	14248.00	-2432.00	111.00	-241.00	-134.00

1	-9804.00	-32768.00	11432.00	-3184.00	-1023.00	-17275.00	131.00

2	-3672.00	-92.00	14032.00	-2432.00	91.00	-245.00	-127.00

3	-2720.00	-4132.00	15664.00	-2416.00	-681.00	-103.00	-119.00

1	-9912.00	-32768.00	11308.00	-3184.00	-1036.00	-17275.00	98.00

2	-3708.00	-148.00	14132.00	-2400.00	111.00	-241.00	-87.00

3	-2632.00	-4092.00	15808.00	-2416.00	-716.00	-67.00	-126.00

1	-9888.00	-32768.00	11084.00	-3232.00	-1061.00	-17285.00	115.00

2	-3704.00	-84.00	14140.00	-2448.00	118.00	-258.00	-121.00

3	-2632.00	-4192.00	15884.00	-2400.00	-714.00	-76.00	-115.00

1	-9960.00	-32768.00	11400.00	-3232.00	-1047.00	-17285.00	97.00

2	-3676.00	-76.00	14268.00	-2432.00	129.00	-244.00	-102.00

3	-2752.00	-4076.00	15768.00	-2400.00	-717.00	-94.00	-118.00

1	-9944.00	-32768.00	11280.00	-3232.00	-1033.00	-17278.00	105.00

2	-3744.00	-116.00	14264.00	-2400.00	82.00	-238.00	-94.00

3	-2688.00	-4220.00	15796.00	-2416.00	-708.00	-86.00	-147.00

1	-9912.00	-32768.00	11348.00	-3216.00	-1022.00	-17282.00	78.00

3	-2632.00	-4104.00	15824.00	-2416.00	-648.00	-77.00	-114.00

1	-9936.00	-32768.00	11396.00	-3200.00	-979.00	-17237.00	91.00

2	-3628.00	-128.00	14040.00	-2448.00	209.00	-212.00	-119.00

3	-2784.00	-4112.00	15920.00	-2400.00	-646.00	-106.00	-87.00

1	-9888.00	-32768.00	11408.00	-3200.00	-1056.00	-17318.00	188.00

2	-3708.00	-192.00	14208.00	-2416.00	226.00	-216.00	-148.00

3	-2672.00	-4132.00	15756.00	-2368.00	-741.00	-105.00	-93.00

1	-9928.00	-32768.00	11336.00	-3200.00	-1053.00	-17297.00	101.00

2	-3764.00	-164.00	14176.00	-2416.00	12.00	-195.00	-105.00

3	-2760.00	-4048.00	15768.00	-2432.00	-695.00	-114.00	-111.00

1	-9836.00	-32768.00	11328.00	-3200.00	-1000.00	-17245.00	149.00

2	-3748.00	-156.00	14108.00	-2416.00	239.00	-227.00	-121.00

3	-2720.00	-4152.00	15712.00	-2432.00	-707.00	-89.00	-121.00

1	-9768.00	-32768.00	11236.00	-3200.00	-1012.00	-17257.00	119.00

2	-3788.00	-124.00	14120.00	-2448.00	73.00	-291.00	-64.00

3	-2644.00	-4140.00	15808.00	-2416.00	-726.00	-80.00	-106.0

...

1	-9824.00	-32768.00	11348.00	-3200.00	-1017.00	-17283.00	100.00

2	-3756.00	-128.00	14172.00	-2448.00	124.00	-257.00	-127.00

3	-2672.00	-4120.00	15812.00	-2416.00	-705.00	-75.00	-99.00

1	-9880.00	-32768.00	11304.00	-3200.00	-1041.00	-17264.00	134.00

2	-3740.00	-112.00	14212.00	-2416.00	120.00	-257.00	-104.00

3	-2656.00	-4040.00	15776.00	-2448.00	-721.00	-87.00	-126.00

1	-9824.00	-32768.00	11264.00	-3200.00	-1028.00	-17289.00	134.00

2	-3796.00	-28.00	14164.00	-2416.00	126.00	-251.00	-116.00

3	-2784.00	-4080.00	15724.00	-2400.00	-711.00	-79.00	-124.00

1	-9888.00	-32768.00	11284.00	-3200.00	-1032.00	-17286.00	103.00

2	-3724.00	-112.00	14280.00	-2432.00	123.00	-242.00	-113.00

3	-2768.00	-4016.00	15784.00	-2400.00	-692.00	-101.00	-67.00

1	-9860.00	-32768.00	11224.00	-3168.00	-1034.00	-17279.00	83.00

2	-3708.00	-12.00	14268.00	-2432.00	112.00	-227.00	-111.00

3	-2756.00	-4160.00	15692.00	-2416.00	-708.00	-103.00	-107.00

1	-9912.00	-32768.00	11412.00	-3168.00	-1014.00	-17269.00	

3	-2724.00	-4064.00	15908.00	-2416.00	-690.00	-90.00	-111.00

1	-9880.00	-32768.00	11328.00	-3184.00	-1024.00	-17280.00	117.00

2	-3636.00	-140.00	14100.00	-2432.00	115.00	-247.00	-118.00

3	-2676.00	-4144.00	15740.00	-2416.00	-685.00	-91.00	-115.00

1	-9884.00	-32768.00	11252.00	-3184.00	-1043.00	-17285.00	102.00

2	-3760.00	-264.00	14144.00	-2464.00	102.00	-254.00	-108.00

3	-2720.00	-4064.00	15732.00	-2416.00	-694.00	-93.00	-113.00

1	-9904.00	-32768.00	11392.00	-3184.00	-1031.00	-17281.00	95.00

2	-3876.00	-80.00	14232.00	-2416.00	109.00	-239.00	-115.00

3	-2656.00	-4128.00	15680.00	-2416.00	-717.00	-64.00	-114.00

1	-9848.00	-32768.00	11308.00	-3184.00	-1019.00	-17274.00	130.00

2	-3832.00	-156.00	14120.00	-2416.00	121.00	-228.00	-112.00

3	-2664.00	-4120.00	15724.00	-2432.00	-703.00	-84.00	-109.00

1	-9892.00	-32768.00	11324.00	-3184.00	-1011.00	-17268.00	126.00

2	-3768.00	-108.00	14132.00	-2448.00	89.00	-245.00	-94.00

3	-2724.00	-4232.00	15792.00	-2432.00	-705.00	-95.00	-121.0

2	-3736.00	-236.00	14032.00	-2432.00	113.00	-229.00	-119.00

3	-2780.00	-4184.00	15844.00	-2368.00	-696.00	-98.00	-98.00

1	-10008.00	-32768.00	11308.00	-3200.00	-1046.00	-17272.00	110.00

2	-3892.00	-112.00	14084.00	-2432.00	110.00	-247.00	-105.00

3	-2732.00	-4132.00	15860.00	-2432.00	-692.00	-80.00	-114.00

1	-10016.00	-32768.00	11328.00	-3168.00	-1048.00	-17296.00	132.00

2	-3820.00	-180.00	14208.00	-2432.00	138.00	-241.00	-125.00

3	-2728.00	-4096.00	15772.00	-2400.00	-714.00	-80.00	-116.00

1	-9940.00	-32768.00	11272.00	-3152.00	-1037.00	-17273.00	110.00

2	-3880.00	-92.00	14292.00	-2400.00	123.00	-241.00	-130.00

3	-2680.00	-4132.00	15732.00	-2432.00	-721.00	-88.00	-81.00

1	-9888.00	-32768.00	11244.00	-3168.00	-1021.00	-17266.00	116.00

2	-3668.00	-136.00	14196.00	-2416.00	124.00	-233.00	-112.00

3	-2708.00	-4112.00	15752.00	-2416.00	-702.00	-79.00	-111.00

1	-9960.00	-32768.00	11360.00	-3168.00	-1044.00	-17287.00	82.00

2	-3752.00	-52.00	14156.00	-2416.00	139.00	-229.00	-124.0

2	-3684.00	-92.00	14096.00	-2448.00	115.00	-263.00	-128.00

3	-2732.00	-4056.00	15728.00	-2416.00	-699.00	-94.00	-80.00

1	-9832.00	-32768.00	11348.00	-3184.00	-1053.00	-17264.00	107.00

2	-3724.00	-44.00	14264.00	-2448.00	99.00	-252.00	-89.00

3	-2644.00	-4120.00	15876.00	-2448.00	-740.00	-109.00	-86.00

1	-9812.00	-32768.00	11404.00	-3200.00	-1037.00	-17272.00	87.00

2	-3664.00	-140.00	14180.00	-2448.00	91.00	-265.00	-119.00

3	-2720.00	-4224.00	15740.00	-2416.00	-673.00	-108.00	-93.00

1	-9856.00	-32768.00	11256.00	-3184.00	-1082.00	-17286.00	119.00

2	-3716.00	-148.00	14064.00	-2400.00	244.00	-251.00	-142.00

3	-2704.00	-4104.00	15836.00	-2400.00	-689.00	-91.00	-126.00

1	-9892.00	-32768.00	11332.00	-3184.00	-1084.00	-17333.00	108.00

2	-3808.00	-212.00	14396.00	-2416.00	36.00	-251.00	-93.00

3	-2752.00	-4052.00	15892.00	-2432.00	-659.00	-85.00	-135.00

1	-9968.00	-32768.00	11220.00	-3184.00	-1073.00	-17326.00	144.00

2	-3748.00	-168.00	14152.00	-2400.00	202.00	-184.00	-114.00

3	-

KeyboardInterrupt: 

In [25]:
1/0.002352

425.1700680272109

In [14]:
1/0.0123

81.30081300813008

In [22]:
1/400

0.0025